# OS11 - Exercises 0-6

In [20]:
from sympy import *

In [21]:
import numpy as np

In [22]:
import scipy.linalg as linalg

In [79]:
from scipy.optimize import linprog

In [23]:
from itertools import *

## Exercise 0

Build a function that transforms a Node-Node matrix to a Node-Arc matrix.

### Data Introduction

Data will be introduced on a list of (origin, dest, cost) tuples (or any indexable).

In [24]:
datos_originales = []

In [25]:
datos_originales.append((1, 2, 1))
#datos_originales.append((2, 1, 1))   #creo un ciclo a proposito #no es una buena idea
datos_originales.append((1, 3, 1))
datos_originales.append((2, 3, 1))
datos_originales.append((3, 4, 1))
datos_originales.append((4, 1, 1))


In [26]:
datos_originales

[(1, 2, 1), (1, 3, 1), (2, 3, 1), (3, 4, 1), (4, 1, 1)]

### Node-Node Matrix creation

We need to pass from a sparse to a dense matrix formulation.

In [27]:
#Max nodes:
nodes = 0
for d in datos_originales:
    if d[0]> nodes:
        nodes =d[0]
    if d[1] > nodes:
        nodes = d[1]

In [28]:
nodes

4

In [29]:
NN = np.zeros((4,4))

In [30]:
NN

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [31]:
for d in datos_originales:
    NN[d[0]-1, d[1]-1] = d[2]

In [32]:
NN

array([[0., 1., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

### Arc Detection

We create a funtion that returns a list of arcs and weights from NN matrix. Basically reverts to original data.

In [33]:
def arc_detection(NN):
    arcs = []
    #each arc will be represented as a tuple
    for a in ((x,y) for x in range(NN.shape[0]) for y in range(NN.shape[1])):
        if NN[a]>0:
            arcs.append((a, NN[a]))
    return arcs

In [34]:
arcs = arc_detection(NN)

In [35]:
arcs

[((0, 1), 1.0), ((0, 2), 1.0), ((1, 2), 1.0), ((2, 3), 1.0), ((3, 0), 1.0)]

### Arc-Nodes Matrix Creation

Finally we created ANMatrix funtion that receives a n N x N matrix and returns a Nodes x Arcs Matrix.

In [36]:
def ANMatrix(NN):
    arcs = arc_detection(NN)
    MN = np.zeros((NN.shape[0], len(arcs))).astype(int)
    for a in range(len(arcs)):
        MN[arcs[a][0][0], a] = arcs[a][1]
        MN[arcs[a][0][1], a] = -arcs[a][1]
    arcss = [a[0] for a in arcs]
    return MN, arcss

In [37]:
ANMatrix(NN)

(array([[ 1,  1,  0,  0, -1],
        [-1,  0,  1,  0,  0],
        [ 0, -1, -1,  1,  0],
        [ 0,  0,  0, -1,  1]]),
 [(0, 1), (0, 2), (1, 2), (2, 3), (3, 0)])

## Ex. 01-02

Data introduction: we load the graph using Ex. 0 technique.

In [38]:
named_nodes = ['s', 2, 3, 4, 5, 't']   #node enumeration. Can be any non repeated inmutable.

I want to use index notation on node names, for this we can create a dictionary with node names and indexes.

In [39]:
def nodes_indexer(named_nodes):
    nodes = {}
    for i in range(len(named_nodes)):
        nodes[named_nodes[i]] = i
    return nodes

In [40]:
nodes = nodes_indexer(named_nodes)

Example:

In [41]:
nodes['t']

5

In [42]:
nodes[5]

4

Using nodes indexed, I load oriented graph data using Ex. 0 schema.

In [43]:
graph = []

graph.append(((nodes['s'], nodes[2]), 2))
graph.append(((nodes['s'], nodes[3]), 2))
graph.append(((nodes[2], nodes[4]), 2))
graph.append(((nodes[2], nodes['t']), 5))
graph.append(((nodes[3], nodes[5]), 2))
graph.append(((nodes[4], nodes['t']), 1))
graph.append(((nodes[5], nodes['t']), 2))

graph


[((0, 1), 2),
 ((0, 2), 2),
 ((1, 3), 2),
 ((1, 5), 5),
 ((2, 4), 2),
 ((3, 5), 1),
 ((4, 5), 2)]

I'll construct a new Arc-Nodes matrix function that uses directly graph data.

In [54]:
def ANMatrix_from_arcs(graph, num_nodes):
    num_arcs = len(graph)
    arcs = []
    MN = np.zeros((num_nodes, num_arcs)).astype(int)
    for a in range(len(graph)):
        MN[graph[a][0][0], a] = 1
        MN[graph[a][0][1], a] = -1
        arcs.append(graph[a][0])
    return MN, arcs

In [57]:
res = ANMatrix_from_arcs(graph, len(nodes))
res

(array([[ 1,  1,  0,  0,  0,  0,  0],
        [-1,  0,  1,  1,  0,  0,  0],
        [ 0, -1,  0,  0,  1,  0,  0],
        [ 0,  0, -1,  0,  0,  1,  0],
        [ 0,  0,  0,  0, -1,  0,  1],
        [ 0,  0,  0, -1,  0, -1, -1]]),
 [(0, 1), (0, 2), (1, 3), (1, 5), (2, 4), (3, 5), (4, 5)])

In [58]:
MAN = res[0]
arcos = res[1]

Necesitamos construir el vector de costos (longitudes entre nodos).

In [63]:
def costos(graph):
    c = []
    for a in graph:
        c.append(a[1])
    return c

In [69]:
c = costos(graph)
c = np.array(c)
c

array([2, 2, 2, 5, 2, 1, 2])

El verctor de igualdades b:
    

In [76]:
b = np.zeros((len(nodes)))
b

array([0., 0., 0., 0., 0., 0.])

Asignamos los valores predeterminados de los nodos de entrada y salida:

In [77]:
b[nodes['s']] = 1
b[nodes['t']] = -1

In [78]:
b

array([ 1.,  0.,  0.,  0.,  0., -1.])

In [81]:
limites = [(0, None) for a in arcos]

In [82]:
limites

[(0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None)]

In [84]:
r = linprog(c=c, A_eq = MAN, b_eq = b, bounds = limites)

<ipython-input-84-41428d71e259>:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  r = linprog(c=c, A_eq = MAN, b_eq = b, bounds = limites)


In [100]:
resultados = r['x']

"Limamos" los resultados:

In [101]:
resultados = resultados.round(3)

In [102]:
resultados

array([1., 0., 1., 0., 0., 1., 0.])

El costo total:

In [104]:
r['fun']

5.000000001088329

Los arcos seleccionados:

In [105]:
camino = []
for a in range(len(graph)):
    if resultados[a] > 0:
        camino.append(graph[a])

In [106]:
camino

[((0, 1), 2), ((1, 3), 2), ((3, 5), 1)]

In [115]:
def print_camino(camino):
    s = ''
    sep = ' -> '
    if len(camino)>0:
        s += str(camino[0][0][0]) + sep
    for t in camino:
        s += str(t[0][1]) + sep
    return s

In [116]:
print_camino(camino)

'0 -> 1 -> 3 -> 5 -> '

True